In [1]:
import os
import urllib.request
import tarfile

# Define the URL and download location
dataset_url = "http://madm.dfki.de/files/sentinel/EuroSAT.zip"
dataset_dir = "EuroSAT"

# Download and extract the dataset
if not os.path.exists(dataset_dir):
    os.mkdir(dataset_dir)
    urllib.request.urlretrieve(dataset_url, os.path.join(dataset_dir, "EuroSAT.zip"))
    with tarfile.open(os.path.join(dataset_dir, "EuroSAT.zip"), "r:gz") as tar:
        tar.extractall(path=dataset_dir)

print("EuroSAT dataset downloaded and extracted.")

EuroSAT dataset downloaded and extracted.


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define paths
data_dir = "C:/Users/User/OneDrive/Desktop/LULC/data/data/2750"

# Load and preprocess images
def load_data(data_dir):
    categories = os.listdir(data_dir)
    data = []
    labels = []
    for category in categories:
        category_path = os.path.join(data_dir, category)
        for file in os.listdir(category_path):
            file_path = os.path.join(category_path, file)
            image = tf.keras.preprocessing.image.load_img(file_path, target_size=(64, 64))
            image = tf.keras.preprocessing.image.img_to_array(image)
            data.append(image)
            labels.append(category)
    return np.array(data), np.array(labels)

# Load data
X, y = load_data(data_dir)

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

print("Success")

Success


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from PIL import Image
import numpy as np
import os

# Set data directory
data_dir = "C:/Users/User/OneDrive/Desktop/LULC/data/data/2750"

# Resize images
def load_data(data_dir, size=(64, 64)):
    categories = os.listdir(data_dir)
    data = []
    labels = []
    for category in categories:
        category_path = os.path.join(data_dir, category)
        for file in os.listdir(category_path):
            file_path = os.path.join(category_path, file)
            image = Image.open(file_path)
            image = image.resize(size)
            image = np.array(image)
            data.append(image)
            labels.append(category)
    return np.array(data), np.array(labels)

# Load data
X, y = load_data(data_dir)

# Encode labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

# Flatten images for Random Forest
nsamples, nx, ny, nchannels = X_train.shape
X_train_flat = X_train.reshape((nsamples, nx * ny * nchannels))

nsamples, nx, ny, nchannels = X_test.shape
X_test_flat = X_test.reshape((nsamples, nx * ny * nchannels))

# Train Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train_flat, y_train)

# Predict
y_pred = model.predict(X_test_flat)

# Evaluate the model
print(classification_report(y_test, y_pred, target_names=encoder.classes_))
print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


In [ ]:
import joblib

# Save the model
joblib.dump(model, "lulc_model.joblib")

# Load the model
model = joblib.load("lulc_model.joblib")
